In [42]:
import os
os.system('sudo pip3 install keras')
os.system('sudo pip3 install tensorflow')
os.system('sudo pip3 install biopython')
os.system('sudo apt install --reinstall python*-decorator')
import os
import pandas as pd
import matplotlib.pyplot as plt  
import numpy as np
from keras.layers import Input, Dense, Dropout, concatenate, LSTM, TimeDistributed, Reshape, Flatten, Bidirectional
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras import initializers
from IPython.display import clear_output 
import math
from scipy.signal import hilbert
from scipy import signal
import random


from hyperopt import fmin, Trials, tpe, hp

def build_generator(params, t_steps, seq_dim, ran_dim):
    '''Nice simple generator    
    '''
    model_input = Input(shape=(t_steps, seq_dim,))
    noise = Input(shape=(t_steps, ran_dim,))
    if params['noise']:        
        dat = concatenate([noise, model_input]) 
    else:
        dat = model_input
    for i in range(params['gen_depth']):
        if i == 0:
            if params['bid']:
                gen_rnn = Bidirectional(LSTM(params['gen_width'], return_sequences=True,))(dat)
            else:
                gen_rnn = LSTM(params['gen_width'], return_sequences=True,)(model_input)
        else:
            if params['bid']:
                gen_rnn = Bidirectional(LSTM(params['gen_width'], return_sequences=True,))(gen_rnn)
            else:
                gen_rnn = LSTM(params['gen_width'], return_sequences=True,)(gen_rnn)

    img = TimeDistributed(Dense(1, activation='sigmoid'))(gen_rnn)  
    img_r = Flatten()(img)
    
         
    return Model([noise, model_input], img_r)
  

def build_discriminator(params, t_steps, seq_dim):
    ''' Nice simple discriminator'''
    img = Input(shape=(t_steps,))
    img_r = Reshape((t_steps, 1))(img)
    label = Input(shape=(t_steps, seq_dim,))
    model_input = concatenate([img_r, label]) 
    
    for i in range(params['dis_depth']):
        if i == 0:
            if params['bid']:
                dis_rnn = Bidirectional(LSTM(params['dis_width'], return_sequences=True,))(model_input)
            else:
                dis_rnn = LSTM(params['dis_width'], return_sequences=True,)(model_input)
        else:
            if params['bid']:
                dis_rnn = Bidirectional(LSTM(params['dis_width'], return_sequences=True,))(dis_rnn)
            else:
                dis_rnn = LSTM(params['dis_width'], return_sequences=True,)(dis_rnn)

    disc = TimeDistributed(Dense(1, activation='sigmoid'))(dis_rnn)   
    pt_calls = Flatten()(disc)
    
    return Model([img,label], pt_calls)
    
def output_plt(gloss, dloss):
    '''
    plot out the data
    '''
    clear_output()
    print('gloss: %i dloss: %i' % (gloss, dloss))

    #Save the models
    ind = 0
    plt.rcParams['figure.figsize'] = [10, 25]
    for i in range(0, len(curve_mat), 2):
        #Parse out the data
        curves = curve_mat[[i, i+1]]
        seqs = seq_mat[[i, i+1]]
        
        #Get the noise
        noise = np.random.normal(0,1, size = [2, t_steps, ran_dim])
        
        #Generate some images
        gen_images = generator.predict([noise, seqs])


        #Get images for the discriminator
        X = np.concatenate([curves, gen_images])
        
        #Plot out the data
        plt.subplot((len(curve_mat)/2)/3,3,ind)
        for c in range(len(X)):
            labels = ['Real', 'Real','Fake','Fake']
            labels = [x+'_'+id_mat[i] for x in labels]
            colors = ['red','red','blue','blue']
            plt.plot(temps, X[c], label=labels[c],color=colors[c])
        
        ind +=1
    plt.ylabel('Dissociation')
    plt.legend(loc="upper right")
    plt.xlabel('Temperature (C)')
    plt.ylabel('Dissociation')    
    plt.tight_layout()
    #plt.savefig('gan_%i_melt.pdf' % e, bbox_inches='tight')
    plt.show()

    
    
def objective(params):
    print(params)
    #Run the GAN
    adam = Adam(lr=params['lr'], beta_1=0.5)
    t_steps = curve_mat.shape[1]
    seq_dim = seq_mat.shape[2]
    ran_dim = params['noise_len']
  
    #Build the model
    generator = build_generator(params, t_steps, seq_dim, ran_dim)
    generator.compile(loss='binary_crossentropy', optimizer=adam)
    discriminator=build_discriminator(params, t_steps, seq_dim)
    discriminator.compile(loss='binary_crossentropy', optimizer=adam)
    discriminator.trainable = False
    noise = Input(shape=(t_steps, ran_dim,))
    label = Input(shape=(t_steps, seq_dim,))
    x =generator([noise, label])
           
    ganOutput = discriminator([x, label])    
    gan = Model(inputs=[noise,label], outputs=ganOutput)
    
    gan.compile(loss='binary_crossentropy', optimizer=adam)
    
    #Train
    btch_cnt = math.ceil(len(seq_mat)/params['batch_size'])
    inds = list(range(len(seq_mat)))
    seq_mat_c = np.copy(seq_mat)
    for e in range(1):               
        random.shuffle(inds)
        
        batches = np.array_split(inds, btch_cnt)
        
        for batch in batches:
            np.random.shuffle(seq_mat_c)
            
            #Generate images
            noise = np.random.normal(0,1, size = [len(batch), t_steps, ran_dim])
            gen_images = generator.predict([noise, seq_mat[batch]])
     
            #Labels                 
            y_dis = np.random.uniform(low=0.7, high=1.0, size=(len(batch), t_steps))

             # Train discriminator real
            discriminator.trainable = True
            dloss = discriminator.train_on_batch([curve_mat[batch], seq_mat[batch]], y_dis)

            # Train disc fake
            y_dis = np.random.uniform(low=0.0, high=0.3, size=(len(batch), t_steps))
            dloss = discriminator.train_on_batch([gen_images, seq_mat[batch]], y_dis)
            
            if params['mislabeled']:
                # Train disc mislabeled
                y_dis = np.random.uniform(low=0.0, high=0.3, size=(len(batch), t_steps))
                dloss = discriminator.train_on_batch([curve_mat[batch], seq_mat_c[batch]], y_dis)

            # Train generator
            noise = np.random.normal(0,1, size = [len(batch), t_steps, ran_dim])        
            y_gen = np.random.uniform(low=0.7, high=1.0, size=(len(batch), t_steps))
            discriminator.trainable = False
            
            gloss = gan.train_on_batch([noise, seq_mat[batch]], y_gen)  

                
    #Evaluate
    dist = []
    for i in range(len(curve_mat)):
        
        noise = np.random.normal(0,1, size = [1, t_steps, ran_dim])
        gen_images = generator.predict([noise, np.array([seq_mat[i]])])


        d_cm = np.diff(curve_mat[i])
        d_gi = np.diff(gen_images)
        dist.append(calc_distance(d_cm, d_gi))
    
    mean_dist = sum(dist)/len(dist)
    print(mean_dist)
    output_plt(gloss, dloss)
    return mean_dist
            
def calc_distance(target,test):
    '''
    Distance between vectors
    ''' 
    #target,test = new_shift(target,test)
    target = signal.detrend(target)
    test = signal.detrend(test)
    target_h = np.imag(hilbert(target))
    test_h = np.imag(hilbert(test))
    #convert to numpy arrays
    target = np.array(test)
    test = np.array(test)
    target_h = np.array(target_h)
    test_h = np.array(test_h)
    return (np.linalg.norm(target-test)
        + np.linalg.norm(target_h-test_h))

seq_mat = np.load('seq_mat.npy')
curve_mat = np.load('curve_mat.npy')
id_mat = np.load('id_mat.npy')


space = {
    'noise': hp.choice('noise', [True, False]),
    'gen_depth': hp.choice('gen_depth', list(range(1,3))),
    'gen_width': hp.choice('gen_width', [64,128]),
    'dis_depth': hp.choice('dis_depth', list(range(1,3))),
    'dis_width': hp.choice('dis_width', [64,128]),
    'batch_size': hp.choice('batch_size', list(range(1,len(seq_mat)+1))),
    'bid': hp.choice('bid', [True, False]),
    'mislabeled': hp.choice('mislabeled', [True, False]),
    'epochs':hp.choice('epochs',list(range(1000, 11000, 1000))),
    'lr': hp.choice('lr', [0.2, 0.02, 0.002, 0.0002, 0.00002, 0.000002]),
    'noise_len':hp.choice('noise_len', list(range(1,200)))
}

# Create a trials object
tpe_trials = Trials()
tpe.suggest

tpe_best = fmin(fn=objective, space=space, 
                algo=tpe.suggest, trials=tpe_trials, 
                max_evals=20)






gloss: 0 dloss: 0


NameError: name 't_steps' is not defined

In [1]:
list(range(1000, 11000, 1000))

[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

In [12]:
a = [2,3,5,6]

In [13]:
import random


In [14]:
random.shuffle(a)

In [15]:
a

[3, 6, 2, 5]

In [38]:
np.array([seq_mat[0]]).shape

(1, 150, 215)